In [1]:
import os
from IPython.display import clear_output


# notebook_dir = "/home/balabaevvl/courses/project/FITS/"
notebook_dir = "/home/balabaevvl/courses/project/FITS/src/"
os.chdir(notebook_dir)

GPUs = [
    "GPU-e83bd31b-fcb9-b8de-f617-2d717619413b",
    "GPU-5a9b7750-9f85-49a5-3aae-fe07b1b7661d",
    "GPU-fe2d8dfd-06f2-a5c4-a7fd-4a5f23947005",
    "GPU-0c320096-21ee-4060-8731-826ca2febfab",
    "GPU-baef952c-6609-aace-3b78-e4e07788d5de",
    "GPU-3979d65b-c238-4e9c-0c1c-1aa3f05c56a1",
    "GPU-6c76a2c5-5375-aa06-11d4-0fddfac30e91",
]
os.environ["CUDA_VISIBLE_DEVICES"] = f"{GPUs[1]}"

In [2]:
from fits.data.download import DownloadDatasetAirQuality

DownloadDatasetAirQuality()

In [3]:
import pandas as pd
from fits.config import DatasetsPaths

df = pd.read_csv(
    DatasetsPaths.pm25.value,
    index_col="datetime",
    parse_dates=True,
)

In [4]:
from fits.data.dataset import ModelMode, DatasetAirQuality

dataset = DatasetAirQuality(ModelMode.train)

for sample in dataset:
    break

sample

ForecastingData(observed_data=tensor([[138.,  89., 105.,  ...,  94., 112., 109.],
        [124.,  85., 121.,  ..., 101., 123., 114.],
        [127.,  88., 130.,  ..., 112., 143., 126.],
        ...,
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.]]), observed_mask=tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), time_points=tensor([[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        [ 2.,  2.,  2.,  ...,  2.,  2.,  2.],
        ...,
        [45., 45., 45.,  ..., 45., 45., 45.],
        [46., 46., 46.,  ..., 46., 46., 46.],
        [47., 47., 47.,  ..., 47., 47., 47.]]), feature_ids=tensor([[ 0.,  1.,  2.,  ..., 33., 34., 35.],
        [ 0.,

In [2]:
from fits.modelling.framework import Train
from fits.data.dataset import DatasetAirQuality
from fits.modelling.CSDI.adapter import CSDIAdapter
from fits.data.dataloader import ForecastingDataLoader

train_loader, valid_loader, test_loader = ForecastingDataLoader(
    DatasetAirQuality, batch_size=128
)

csdi = CSDIAdapter()

/home/balabaevvl/courses/venv311/lib/python3.11/site-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [3]:
Train(csdi, train_loader, valid_loader)

  0%|                                                                                                                                                                   | 0/48 [00:00<?, ?it/s]


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'fits.data.dataset.ForecastingData'>

In [5]:
# TODO: data normalization